In [65]:
import pandas as pd
import numpy as np

# 載入數據
file_path = '新竹_2021.xls'
data = pd.read_excel(file_path, skiprows=[1])

# 將無效值轉換為 NaN，並將 'NR' 替換為 0 表示無降雨
data.replace({'#                              ': np.nan,
              '*                              ': np.nan,
              'X                              ': np.nan,
              'A                              ': np.nan,
              'NR                              ': 0,
              '': np.nan}, inplace=True)

# 提取日期資訊，並篩選 10 月、11 月和 12 月的數據
data['Date'] = pd.to_datetime(data["日期                  "], errors='coerce')
data_oct_nov = data[(data['Date'].dt.month == 10) | (data['Date'].dt.month == 11)]
data_dec = data[data['Date'].dt.month == 12]

def fill_missing_with_nearest_avg(data):
    for column in data.columns:
        if column not in ['Date', "日期                  ", "測站                  ", "測項                  "]:
            # 將該列轉換為數值型，無法轉換的值設為 NaN
            data[column] = pd.to_numeric(data[column], errors='coerce')
            
            # 遍歷每個數據行並填補 NaN 值
            for i in range(len(data)):
                if pd.isna(data.iloc[i][column]):
                    offset = 1
                    values_to_average = []
                    
                    # 只取前後一個小時的有效數據
                    while not values_to_average and (i - offset >= 0 or i + offset < len(data)):
                        # 向前找數值
                        if i - offset >= 0:
                            prev_value = data.iloc[i - offset][column]
                            if not pd.isna(prev_value):
                                values_to_average.append(prev_value)
                                
                        # 向後找數值
                        if i + offset < len(data):
                            next_value = data.iloc[i + offset][column]
                            if not pd.isna(next_value):
                                values_to_average.append(next_value)
                        
                        offset += 1
                    
                    # 如果找到有效數值，則計算平均值來填補 NaN
                    if values_to_average:
                        data.iloc[i, data.columns.get_loc(column)] = np.mean(values_to_average)
                        
    return data

# 填補缺失值
data_oct_nov = fill_missing_with_nearest_avg(data_oct_nov)
data_dec = fill_missing_with_nearest_avg(data_dec)

# 將 10 月和 11 月的數據設為訓練集，12 月的數據設為測試集
train_data = data_oct_nov
test_data = data_dec

# 製作時序資料
# 重新設定索引，確保數據可以依測項和時間排序
train_data = train_data.sort_values(['Date', '測項                  ']).reset_index(drop=True)

# 將每個屬性的逐時數據排列成單行
train_data_pivot = train_data.pivot(index="測項                  ", columns='Date', values=list(range(24)))

# 將每天的 24 小時展平成單列，使每個測項成為 1464 列
train_data_flat = pd.DataFrame()
for item in train_data_pivot.index:
    daily_values = train_data_pivot.loc[item].values.flatten()  # 展平成單列
    train_data_flat[item] = daily_values

test_data = test_data.sort_values(['Date', '測項                  ']).reset_index(drop=True)

# 將每個屬性的逐時數據排列成單行
test_data_pivot = test_data.pivot(index="測項                  ", columns='Date', values=list(range(24)))

# 將每天的 24 小時展平成單列，使每個測項成為 1464 列
test_data_flat = pd.DataFrame()
for item in test_data_pivot.index:
    daily_values = test_data_pivot.loc[item].values.flatten()  # 展平成單列
    test_data_flat[item] = daily_values    

# 檢查結果
print("轉換後的資料形狀:",  test_data_flat.shape)
test_data_flat.head()

print("轉換後的資料形狀:", train_data_flat.shape)  # 預期形狀應為 (18, 1464)
train_data_flat.head()


C:\Users\User\AppData\Local\Temp\ipykernel_16564\1591987719.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_numeric(data[column], errors='coerce')


轉換後的資料形狀: (744, 18)
轉換後的資料形狀: (1464, 18)


,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16.0,28.0,28.0,0.0,71.0,1.8,2.21,62.0,33.0,0.8,0.7
1,27.8,2.08,0.37,0.12,0.3,14.2,14.6,23.8,28.0,18.0,0.0,81.0,0.6,2.20,62.0,63.0,0.8,0.8
2,27.8,2.07,0.33,0.13,0.6,15.5,16.1,19.1,22.0,20.0,0.0,79.0,1.1,2.20,116.0,83.0,0.9,0.6
3,27.5,2.10,0.31,0.12,0.5,16.3,16.8,18.8,27.0,11.0,0.0,63.0,0.7,2.22,80.0,88.0,1.0,1.1
4,28.8,1.95,0.16,0.03,0.4,5.7,6.2,32.5,20.0,14.0,0.0,59.0,0.2,1.98,70.0,62.0,1.7,1.7


In [68]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb


# 提取 PM2.5 和所有特徵
pm25_train = train_data_flat["PM2.5               "].values
all_data_train = train_data_flat.values
pm25_test = test_data_flat["PM2.5               "].values
all_data_test = test_data_flat.values

# 建立逐時資料特徵和目標
X_pm25_train_1hr = np.array([pm25_train[i:i+6] for i in range(len(pm25_train) - 6)])
y_train_1hr = pm25_train[6:]
X_pm25_train_6hr = np.array([pm25_train[i:i+6] for i in range(len(pm25_train) - 11)])
y_train_6hr = pm25_train[11:]

X_all_train_1hr = np.array([all_data_train[i:i+6].flatten() for i in range(len(pm25_train) - 6)])
X_all_train_6hr = np.array([all_data_train[i:i+6].flatten() for i in range(len(pm25_train) - 11)])

# 標準化訓練資料
scaler_pm25 = StandardScaler()
scaler_all = StandardScaler()
X_pm25_train_1hr = scaler_pm25.fit_transform(X_pm25_train_1hr)
X_pm25_train_6hr = scaler_pm25.transform(X_pm25_train_6hr)
X_all_train_1hr = scaler_all.fit_transform(X_all_train_1hr)
X_all_train_6hr = scaler_all.transform(X_all_train_6hr)

# 準備12月測試集資料
X_pm25_test_1hr = np.array([pm25_test[i:i+6] for i in range(len(pm25_test) - 6)])
y_test_1hr = pm25_test[6:]
X_pm25_test_6hr = np.array([pm25_test[i:i+6] for i in range(len(pm25_test) - 11)])
y_test_6hr = pm25_test[11:]

X_all_test_1hr = np.array([all_data_test[i:i+6].flatten() for i in range(len(pm25_test) - 6)])
X_all_test_6hr = np.array([all_data_test[i:i+6].flatten() for i in range(len(pm25_test) - 11)])

# 使用訓練的標準化器標準化測試資料
X_pm25_test_1hr = scaler_pm25.transform(X_pm25_test_1hr)
X_pm25_test_6hr = scaler_pm25.transform(X_pm25_test_6hr)
X_all_test_1hr = scaler_all.transform(X_all_test_1hr)
X_all_test_6hr = scaler_all.transform(X_all_test_6hr)

# 線性回歸模型
lr_model = LinearRegression()
lr_model.fit(X_all_train_1hr, y_train_1hr)
y_pred_1hr = lr_model.predict(X_all_test_1hr)
mae_1hr = mean_absolute_error(y_test_1hr, y_pred_1hr)
print(f"Linear Regression - MAE for 1hr target: {mae_1hr}")

lr_model.fit(X_all_train_6hr, y_train_6hr)
y_pred_6hr = lr_model.predict(X_all_test_6hr)
mae_6hr = mean_absolute_error(y_test_6hr, y_pred_6hr)
print(f"Linear Regression - MAE for 6hr target: {mae_6hr}")

# XGBoost 模型
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth=7, n_estimators=300)
xgb_model.fit(X_all_train_1hr, y_train_1hr)
y_pred_1hr_xgb = xgb_model.predict(X_all_test_1hr)
mae_1hr_xgb = mean_absolute_error(y_test_1hr, y_pred_1hr_xgb)
print(f"XGBoost - MAE for 1hr target: {mae_1hr_xgb}")

xgb_model.fit(X_all_train_6hr, y_train_6hr)
y_pred_6hr_xgb = xgb_model.predict(X_all_test_6hr)
mae_6hr_xgb = mean_absolute_error(y_test_6hr, y_pred_6hr_xgb)
print(f"XGBoost - MAE for 6hr target: {mae_6hr_xgb}")

# 線性回歸模型
lr_model = LinearRegression()
lr_model.fit(X_pm25_train_1hr, y_train_1hr)
y_pred_1hr = lr_model.predict(X_pm25_test_1hr)
mae_1hr = mean_absolute_error(y_test_1hr, y_pred_1hr)
print(f"Linear Regression - MAE for 1hr target (using only PM2.5): {mae_1hr}")

lr_model.fit(X_pm25_train_6hr, y_train_6hr)
y_pred_6hr = lr_model.predict(X_pm25_test_6hr)
mae_6hr = mean_absolute_error(y_test_6hr, y_pred_6hr)
print(f"Linear Regression - MAE for 6hr target (using only PM2.5): {mae_6hr}")

# XGBoost 模型
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth=7, n_estimators=300)
xgb_model.fit(X_pm25_train_1hr, y_train_1hr)
y_pred_1hr_xgb = xgb_model.predict(X_pm25_test_1hr)
mae_1hr_xgb = mean_absolute_error(y_test_1hr, y_pred_1hr_xgb)
print(f"XGBoost - MAE for 1hr target (using only PM2.5): {mae_1hr_xgb}")

xgb_model.fit(X_pm25_train_6hr, y_train_6hr)
y_pred_6hr_xgb = xgb_model.predict(X_pm25_test_6hr)
mae_6hr_xgb = mean_absolute_error(y_test_6hr, y_pred_6hr_xgb)
print(f"XGBoost - MAE for 6hr target (using only PM2.5): {mae_6hr_xgb}")


Linear Regression - MAE for 1hr target: 6.284552909079935
Linear Regression - MAE for 6hr target: 7.95435410351292
XGBoost - MAE for 1hr target: 5.683229496808556
XGBoost - MAE for 6hr target: 7.257036364582681
Linear Regression - MAE for 1hr target (using only PM2.5): 5.59282529151517
Linear Regression - MAE for 6hr target (using only PM2.5): 5.6257726438263855
XGBoost - MAE for 1hr target (using only PM2.5): 6.441533750634853
XGBoost - MAE for 6hr target (using only PM2.5): 6.342787148031987
